In [72]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [84]:
data = pd.read_csv("./ronaldo_goal_prediction.csv")

data

,Сила суперника,Місце матчу,Форма Роналдо,Підтримка команди,Забив
0,Сильний,В гостях,Добра,Висока,Ні
1,Слабкий,Дома,Добра,Висока,Так
2,Слабкий,В гостях,Погана,Низька,Ні
3,Слабкий,Дома,Добра,Низька,Так
4,Сильний,Дома,Добра,Висока,Так
5,Слабкий,В гостях,Погана,Низька,Ні
6,Слабкий,В гостях,Добра,Висока,Так
7,Слабкий,Дома,Погана,Низька,Ні
8,Сильний,Дома,Добра,Низька,Так
9,Слабкий,В гостях,Погана,Висока,Ні


In [85]:
def Gini(source, x, y, left_split, right_split):
    data = source[[x, y]]
    
    # Лівий split
    left_data = data[data[x].isin(left_split)]
    left_total = len(left_data)
    
    # Правий split
    right_data = data[data[x].isin(right_split)]
    right_total = len(right_data)
    
    # Перевірка на порожні групи
    if left_total == 0 or right_total == 0:
        return None, None, float('inf')
    
    # Створюємо таблицю для лівого split
    print(f"\n{'='*50}")
    print(f"ЛІВИЙ SPLIT: {list(left_split)}")
    print(f"{'='*50}")
    left_table = left_data.groupby(y).size().to_frame(name='Кількість').T
    left_table['Тотал'] = left_total
    print(left_table)
    
    # Створюємо таблицю для правого split
    print(f"\n{'='*50}")
    print(f"ПРАВИЙ SPLIT: {list(right_split)}")
    print(f"{'='*50}")
    right_table = right_data.groupby(y).size().to_frame(name='Кількість').T
    right_table['Тотал'] = right_total
    print(right_table)
    
    # Обчислення Gini
    left_gini = 1.0 - sum((left_data[y].value_counts() / left_total) ** 2)
    right_gini = 1.0 - sum((right_data[y].value_counts() / right_total) ** 2)
    total_gini = (left_total / len(data)) * left_gini + (right_total / len(data)) * right_gini
    
    print(f"\n{'='*50}")
    print(f"GINI ІНДЕКСИ:")
    print(f"{'='*50}")
    print(f"  Лівий Gini:  {left_gini:.4f}")
    print(f"  Правий Gini: {right_gini:.4f}")
    print(f"  Загальний Gini: {total_gini:.4f}")
    print(f"{'='*50}\n")

    return left_gini, right_gini, total_gini

In [86]:
import itertools
import numpy as np

def get_subset(data, x):
    unique = data[x].unique()
    subset = []
    seen = set()

    for i in range(1, len(unique)):
        for combo in itertools.combinations(unique, i):
            left = tuple(sorted(combo))
            right = tuple(sorted(np.setdiff1d(unique, combo)))
            
            pair = tuple(sorted([left, right]))
            
            if pair not in seen:
                seen.add(pair)
                subset.append((np.array(left), np.array(right)))

    return subset

In [87]:
X = data.drop(columns="Забив")

In [88]:
def compute_gini(columns):
    for col in columns:
        subset_list = get_subset(data, col)
    
        best_gini = float('inf')
        best_split = None
        
        # Перевіряємо ВСІ можливі splits
        for left_split, right_split in subset_list:
            left_gini, right_gini, total_gini = Gini(data, col, "Забив", left_split, right_split)
            
            if total_gini is not None and total_gini < best_gini:
                best_gini = total_gini
                best_split = (left_split, right_split, left_gini, right_gini)
        
        if best_split:
            left_split, right_split, left_gini, right_gini = best_split
            print(f"{col}:")
            print(f"  Найкращий split: {left_split} | {right_split}")
            print(f"  Лівий Gini: {left_gini:.4f}")
            print(f"  Правий Gini: {right_gini:.4f}")
            print(f"  Загальний Gini: {best_gini:.4f}")
            print("---------------------------------")

In [89]:
compute_gini(X.columns)


ЛІВИЙ SPLIT: ['Сильний']
Забив      Ні  Так  Тотал
Кількість   2    2      4

ПРАВИЙ SPLIT: ['Слабкий']
Забив      Ні  Так  Тотал
Кількість   4    4      8

GINI ІНДЕКСИ:
  Лівий Gini:  0.5000
  Правий Gini: 0.5000
  Загальний Gini: 0.5000

Сила суперника:
  Найкращий split: ['Сильний'] | ['Слабкий']
  Лівий Gini: 0.5000
  Правий Gini: 0.5000
  Загальний Gini: 0.5000
---------------------------------

ЛІВИЙ SPLIT: ['В гостях']
Забив      Ні  Так  Тотал
Кількість   5    1      6

ПРАВИЙ SPLIT: ['Дома']
Забив      Ні  Так  Тотал
Кількість   1    5      6

GINI ІНДЕКСИ:
  Лівий Gini:  0.2778
  Правий Gini: 0.2778
  Загальний Gini: 0.2778

Місце матчу:
  Найкращий split: ['В гостях'] | ['Дома']
  Лівий Gini: 0.2778
  Правий Gini: 0.2778
  Загальний Gini: 0.2778
---------------------------------

ЛІВИЙ SPLIT: ['Добра']
Забив      Ні  Так  Тотал
Кількість   1    6      7

ПРАВИЙ SPLIT: ['Погана']
Забив      Ні  Тотал
Кількість   5      5

GINI ІНДЕКСИ:
  Лівий Gini:  0.2449
  Правий Gini: 0

In [90]:
data = data[data["Форма Роналдо"] == "Добра"]
data

,Сила суперника,Місце матчу,Форма Роналдо,Підтримка команди,Забив
0,Сильний,В гостях,Добра,Висока,Ні
1,Слабкий,Дома,Добра,Висока,Так
3,Слабкий,Дома,Добра,Низька,Так
4,Сильний,Дома,Добра,Висока,Так
6,Слабкий,В гостях,Добра,Висока,Так
8,Сильний,Дома,Добра,Низька,Так
10,Слабкий,Дома,Добра,Висока,Так


In [91]:
compute_gini(X.drop(columns="Форма Роналдо").columns)


ЛІВИЙ SPLIT: ['Сильний']
Забив      Ні  Так  Тотал
Кількість   1    2      3

ПРАВИЙ SPLIT: ['Слабкий']
Забив      Так  Тотал
Кількість    4      4

GINI ІНДЕКСИ:
  Лівий Gini:  0.4444
  Правий Gini: 0.0000
  Загальний Gini: 0.1905

Сила суперника:
  Найкращий split: ['Сильний'] | ['Слабкий']
  Лівий Gini: 0.4444
  Правий Gini: 0.0000
  Загальний Gini: 0.1905
---------------------------------

ЛІВИЙ SPLIT: ['В гостях']
Забив      Ні  Так  Тотал
Кількість   1    1      2

ПРАВИЙ SPLIT: ['Дома']
Забив      Так  Тотал
Кількість    5      5

GINI ІНДЕКСИ:
  Лівий Gini:  0.5000
  Правий Gini: 0.0000
  Загальний Gini: 0.1429

Місце матчу:
  Найкращий split: ['В гостях'] | ['Дома']
  Лівий Gini: 0.5000
  Правий Gini: 0.0000
  Загальний Gini: 0.1429
---------------------------------

ЛІВИЙ SPLIT: ['Висока']
Забив      Ні  Так  Тотал
Кількість   1    4      5

ПРАВИЙ SPLIT: ['Низька']
Забив      Так  Тотал
Кількість    2      2

GINI ІНДЕКСИ:
  Лівий Gini:  0.3200
  Правий Gini: 0.0000
  Загал

In [92]:
data = data[data["Місце матчу"] != "Дома"]
data

,Сила суперника,Місце матчу,Форма Роналдо,Підтримка команди,Забив
0,Сильний,В гостях,Добра,Висока,Ні
6,Слабкий,В гостях,Добра,Висока,Так


In [93]:
compute_gini(X.drop(columns=["Форма Роналдо", "Місце матчу"]).columns)


ЛІВИЙ SPLIT: ['Сильний']
Забив      Ні  Тотал
Кількість   1      1

ПРАВИЙ SPLIT: ['Слабкий']
Забив      Так  Тотал
Кількість    1      1

GINI ІНДЕКСИ:
  Лівий Gini:  0.0000
  Правий Gini: 0.0000
  Загальний Gini: 0.0000

Сила суперника:
  Найкращий split: ['Сильний'] | ['Слабкий']
  Лівий Gini: 0.0000
  Правий Gini: 0.0000
  Загальний Gini: 0.0000
---------------------------------
